In [ ]:
from torchvision import datasets

from functools import partial
import numpy as np

# preprocessing
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split

# model
import torch
from torch import nn
from torch.optim import AdamW

# hyperparameter tuning
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

# 1. Load Data

In [ ]:
def load_data(data_dir):
    """ Create train and test pytorch dataset objects from CIFAR10.
    
    The following tranformations are applied on CIFAR10:
        * image pixels to [0, 1] range,
        * normalization (by substracting mean and dividing with std; [-1, 1] range)
    
    Args:
        data_dir:
            directory where data will be saved, as a string.
    
    Returns:
        train and test dataset, as pytorch dataset objects.
    """
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = datasets.CIFAR10(root=data_dir,
                                train=True, 
                                download=True, 
                                transform=transform)

    testset = datasets.CIFAR10(root=data_dir, 
                               train=False, 
                               download=True, 
                               transform=transform)

    return trainset, testset

In [ ]:
training_data, test_data = load_data(data_dir='cifar10')

print(f'\nTraining data:\n--------------\n{training_data}')
print(f'Test data:\n--------------\n{test_data}')

# 2. CNN

**Architecture:** Input &#8594; CONV1 &#8594; ReLU &#8594; POOL2 &#8594; CONV3 &#8594; ReLU &#8594; POOL4 &#8594; CONV5 &#8594; ReLU &#8594; FC6 &#8594; Softmax

Specifically, the CNN's convolutional layers are as follows:
* **CONV1** has 6 filters and kernel size of 5 x 5,
* **CONV3** has 16 filters and kernel size of 5 x 5,
* **CONV5** has 120 filters and kernel size of 5 x 5.

Also, the CNN has:
* **max pooling layer** with receptive field of 2 x 2,
* **dropout layer** and **L2 regularization** to avoid overfitting,
* **ReLU** activation function in hidden layers and **Softmax** in output layer.

In [ ]:
class CNN(nn.Module):
    
    def __init__(self, in_channels, output_size, p):
        super(CNN, self).__init__()
        
        self.in_channels = in_channels
        self.output_size = output_size
        self.p = p
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=6, kernel_size=5, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=output_size)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        
        # flatten all dimensions except batch
        x = torch.flatten(x, 1)
        
        logits = torch.relu(self.fc1(x))
        logits = nn.Dropout(self.p)(logits)
        
        logits = torch.relu(self.fc2(logits))
        logits = nn.Dropout(self.p)(logits)
        
        logits = self.fc3(logits)
        
        return logits

# 3. Hyperparameter Tuning

- [x] early stopping

In [ ]:
def tune_fit(config, train_data, in_channels, num_labels, batch_size, num_workers, epochs):
    """ Fit convolutional neural network and hyperparameter tune in validation set.
    
    The validation set is 20% hold out data from the training set. <br>
    The metric used is accuracy score.
    
    Args:
        config:
            hyperparameters of neural network, as a dictionary.
        train_data:
            training dataset, as a pytorch object.
        in_channels:
            number of input channels, as an integer.
        num_labels:
            number of labels, as an integer.
        batch_size:
            size of batches to be processed, as an integer.
        num_workers:
            how many subprocesses to use for data loading (0 means that the data 
            will be loaded in the main process), as an integer.
        epochs:
            number of epochs (times the neural network will see the data), as an integer.
    """
    model = CNN(in_channels=in_channels,
                output_size=num_labels,
                p=config['p'])
    
    # support data parallel training on multiple GPUs
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
    model.to(device)
    
    loss_fn = nn.CrossEntropyLoss()
    
    optimizer = AdamW(model.parameters(), lr=config['lr'], weight_decay=config['l2'])
    
    # split data: training (80%) - validation (20%)
    test_abs = int(len(train_data) * 0.8)
    train_subset, val_subset = random_split(train_data, [test_abs, len(train_data) - test_abs])

    train_dataloader = DataLoader(train_subset,
                                  batch_size=batch_size,
                                  shuffle=True, 
                                  num_workers=num_workers)
    
    val_dataloader = DataLoader(val_subset,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=num_workers)
    
    for epoch in range(epochs):
        model.train()  # put on train mode
        for batch, (X, Y) in enumerate(train_dataloader, 0):
            # send the data to the GPU memory explicitly
            X, Y = X.to(device), Y.to(device)

            # reset the gradients
            optimizer.zero_grad()
            
            # compute prediction
            pred = model(X)

            # compute loss
            loss = loss_fn(pred, Y)

            # backpropagate
            loss.backward()

            # update parameters
            optimizer.step()

        # validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        model.eval()  # put on evaluation mode
        for batch, (X, Y) in enumerate(val_dataloader, 0):
            with torch.no_grad():
                X, Y = X.to(device), Y.to(device)

                pred = model(X)
                
                _, predicted = torch.max(pred.data, 1)
                total += Y.size(0)
                correct += (predicted == Y).sum().item()

                loss = loss_fn(pred, Y)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        tune.report(loss=(val_loss / val_steps), accuracy=(correct / total))

In [ ]:
def hyperparameter_tune(data_dir,
                        batch_size,
                        epochs,
                        config,
                        grace_period,
                        num_samples, 
                        cpus_per_trial,
                        gpus_per_trial):
    """ Run hyperparameter tuning and report best hyperparameters.
    
    Args:
        data_dir:
            directory where data will be saved, as a string.
        batch_size:
            size of batches to be processed, as an integer.
        epochs:
            number of epochs (times the neural network will see the data), as an integer.
        config:
            hyperparametes, as a dictionary.
        grace_period:
            stop trials at least this old in time, as an integer.
        num_samples:
            number of times to sample from the hyperparameter space (if grid_search is provided as an argument,
            the grid will be repeated num_samples of times), as an integer.
        cpus_per_trial:
            CPUs to allocate per trial, as integer.
        gpus_per_trial:
            GPUs to allocate per trial, as integer.
    """
    trainset, testset = load_data(data_dir=data_dir)
    
    in_channels = trainset[0][0].shape[0]
    num_labels = 10
    
    # hyperparameter search space
    cofig = config
    
    # used for early stopping
    scheduler = ASHAScheduler(metric="loss", 
                              mode="min",
                              max_t=epochs,
                              grace_period=grace_period,
                              reduction_factor=2)
    
    reporter = CLIReporter(metric_columns=["loss", "accuracy", "training_iteration"])
    
    # hyperparamet tuning
    result = tune.run(partial(tune_fit, 
                              train_data=trainset,
                              in_channels=in_channels, 
                              num_labels=num_labels,
                              batch_size=batch_size,
                              num_workers=3, 
                              epochs=epochs),
                      resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
                      config=config,
                      num_samples=num_samples,
                      scheduler=scheduler,
                      progress_reporter=reporter)
    
    # report best results
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
    print("Best trial final validation acurracy: {}".format(best_trial.last_result["accuracy"]))

In [ ]:
# hyperparameter search space
config = {
        "lr": tune.grid_search([1e-4, 1e-3]),
        "l2": tune.grid_search([1e-4, 1e-3]),
        "p": tune.grid_search([0.4, 0.5, 0.6])
    }

In [ ]:
hyperparameter_tune(data_dir='cifar10',
                    batch_size=64,
                    epochs=50,
                    config=config,
                    grace_period=5,
                    num_samples=1, 
                    cpus_per_trial=2,
                    gpus_per_trial=0)

# 4. Fit and Predict

In [ ]:
def fit(dataloader, model, loss_fn, optimizer, print_loss=False):
    """ Fit convolutional neural network.
    
    Args:
        dataloader:
            pytorch DataLoader object.
        model:
            convolutional neural network, as pytorch object.
        loss_fn:
            loss function, as pytorch object.
        optimizer:
            optimizer function, as pytorch object.
        print_loss:
            print loss on every batch, as boolean (default False)
    """
    size = len(dataloader.dataset)
    model.train()  # put on train mode
    for batch, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device), Y.to(device)
        
        # compute prediction
        pred = model(X)
        
        # compute loss
        loss = loss_fn(pred, Y)

        # reset the gradients
        optimizer.zero_grad()
        
        # backpropagate
        loss.backward()
        
        # update parameters
        optimizer.step()

        if print_loss and batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def predict(dataloader, model, loss_fn):
    """ Predict with convolutional neural network.
    
    Args:
        dataloader:
            pytorch DataLoader object.
        model:
            convolutional neural network, as pytorch object.
        loss_fn:
            loss function, as pytorch object.
    """
    size = len(dataloader.dataset)
    
    test_loss = 0
    correct = 0
    
    model.eval()  # put on evaluation mode
    with torch.no_grad():
        for X, Y in dataloader:
            X, Y = X.to(device), Y.to(device)
            
            pred = model(X)
            
            test_loss += loss_fn(pred, Y).item()
            
            correct += (pred.argmax(1) == Y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"average loss: {test_loss:>8f} \naccuracy: {(100*correct):>0.1f}%\n")

# 5. Final Model

In [ ]:
batch_size = 64

training_data, test_data = load_data(data_dir='cifar10')

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

In [ ]:
in_channels = 3
num_labels = 10
p=0.4

model = CNN(in_channels=in_channels, output_size=num_labels, p=p)
model.to(device)

print(f'Model architecture:\n{model}')

In [ ]:
learning_rate = 0.001
weight_decay = 0.001
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
epochs = 20

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    fit(train_dataloader, model, loss_fn, optimizer, print_loss=False)
    
    print('\nTrain:\n-------')
    predict(train_dataloader, model, loss_fn)
    
    print('\nTest:\n-------')
    predict(test_dataloader, model, loss_fn)
    
    print(f"-------------------------------")